In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import pyarrow.parquet as pq
import s3fs
import io
import tensorflow as tf
import base64
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models

2025-10-01 20:45:36.456184: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
!aws s3 mb s3://vizdoom-aai530

make_bucket: vizdoom-aai530


In [14]:
!pip install s3fs

In [3]:
fs = s3fs.S3FileSystem()
parquet_file = pq.ParquetFile('s3://vizdoom-aai530/dataset.parquet', filesystem=fs)

table = parquet_file.read_row_groups([0])
df = table.to_pandas()

In [7]:
MAX_SEQ_LENGTH = 20  # pad sequences to 20 steps
X_sequences = pad_sequences(df['health'].tolist(), maxlen=MAX_SEQ_LENGTH, dtype='float32', padding='pre', truncating='pre')
y_sequences = pad_sequences(df['actions'].tolist(), maxlen=MAX_SEQ_LENGTH, dtype='int32', padding='pre', truncating='pre')

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# Compute number of unique actions and class weight
all_actions = np.unique(np.concatenate(df['actions'].values))
num_actions = len(all_actions)

flat_y = y_train.flatten()
class_weights = compute_class_weight(class_weight='balanced', classes=all_actions, y=flat_y)
class_weight_dict = dict(zip(all_actions, class_weights))
print("Class weights:", class_weight_dict)

# build LSTM
model = Sequential([
    Masking(mask_value=0., input_shape=(MAX_SEQ_LENGTH, 1)),  # mask padded zeros
    LSTM(64, return_sequences=True),
    Dense(num_actions, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Reshape for LSTM
X_train_reshaped = X_train[..., np.newaxis]
X_test_reshaped = X_test[..., np.newaxis]

# train
model.fit(
    X_train_reshaped,
    y_train,
    validation_data=(X_test_reshaped, y_test),
    epochs=20,
    batch_size=32,
)

Class weights: {0: 0.10558129099523564, 1: 26.143790849673202, 2: 2.0248038471273095, 3: 12.012012012012011, 4: 4.153686396677051, 5: 4.134366925064599, 6: 1.8214936247723132, 7: 0.43423980893448405, 8: 1.5987210231814548, 9: 0.7168458781362007, 10: 68.37606837606837, 11: 3.1298904538341157, 12: 4.378762999452655, 13: 1.8066847335140017, 14: 1.848001848001848, 15: 3.1189083820662766, 16: 2.9728725380899292, 17: 4.09626216077829}


/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_2 (Masking)             │ (None, 20, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 20, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 20, 18)         │         1,170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,066 (70.57 KB)

 Trainable params: 18,066 (70.57 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.2325 - loss: 2.7363 - val_accuracy: 0.1145 - val_loss: 2.6067
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1301 - loss: 2.5197 - val_accuracy: 0.1145 - val_loss: 2.5399
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1236 - loss: 2.4998 - val_accuracy: 0.1145 - val_loss: 2.5258
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1263 - loss: 2.4702 - val_accuracy: 0.1145 - val_loss: 2.5194
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1327 - loss: 2.4479 - val_accuracy: 0.1145 - val_loss: 2.5171
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1282 - loss: 2.4547 - val_accuracy: 0.1145 - val_loss: 2.5164
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1261 - loss: 2.4636 - val_accuracy: 0.1145 - val_loss: 2.5151
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1291 - loss: 2.4462 - val_accuracy: 0.1163 - v

In [8]:
### Not increasing accuracy due to the exclusion of images so random guessing - training outside of Sagemaker for better results